In [1]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_chroma import Chroma
from langchain_community.llms import Ollama
import chromadb
from pathlib import Path
import os

In [2]:
# Set the absolute path for the Chroma database
ABS_PATH = Path().resolve().joinpath('Chroma')
DB_DIR = os.path.join(ABS_PATH, "env_policy")
print('DB_DIR:', DB_DIR)

DB_DIR: /Users/rahul/Projects/AI-Project/ClimateActionPolicy-RAG/Chroma/env_policy


In [3]:
# Define client settings for Chroma
client_settings = chromadb.config.Settings(
    is_persistent=True,
    persist_directory=DB_DIR,
    anonymized_telemetry=False,
)

In [4]:
# Initialize the HuggingFaceBgeEmbeddings with the correct model
embedder = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Load the Chroma vector store
bge_vectorstore = Chroma(
    embedding_function=embedder,
    client_settings=client_settings,
    collection_name="env_policy_bge",
    collection_metadata={"hnsw": "cosine"}
)

In [6]:
# Initialize the retriever
retriever = bge_vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "include_metadata": True}
)

In [7]:
# Define the query
query = "Give me the top three things to consider when writing my climate action plan."

# Retrieve documents
retrieved_docs = retriever.invoke(query)
retrieved_docs

[Document(page_content='Title: Toolkit to Declare a Climate Emergency\nContent: This short toolkit is designed to help cities declare a climate emergency. It outlines three key steps to declaring a climate emergency, and includes links to additional resources under each step:\nPass declaration(s) of Climate Emergency with commitment to reach zero emissions and begin decarbonization at emergency speed (within 10 years).\nLocal elected leaders become advocates for emergency climate mobilisation to the public, media, and other cities.\nDevelop and implement mobilisation policies locally, after the declaration has been passed.\nStep three incorporates developing and implementing a climate action plan. For more comprehensive guidance on how to do this, visit the Climate Action Planning section of this website.\nThe toolkit includes a template press release which cities can adapt, key messages for use in public communications, and common questions and answers. It also provides an alternative

In [24]:
import subprocess
import json

# Function to create a context from retrieved documents
def create_context_from_docs(docs):
    return " ".join([doc.page_content for doc in docs])

# Create context from retrieved documents
context = create_context_from_docs(retrieved_docs)





In [25]:
# Define the question
question = "Give me the top three things to consider when writing my climate action plan."


In [26]:
# Function to generate an answer using the local Ollama CLI
llm = Ollama(model = 'llama3')
def generate_with_ollama3(context, question):
    input_text = f"{context} {question}"
    # Use subprocess to run the Ollama model
    result = llm.invoke(input_text)
    return result

# Generate the answer using the context and the question
answer = generate_with_ollama3(context, question)

print("Answer:", answer)

Answer: Based on the toolkit, here are the top three things to consider when writing your climate action plan:

1. **Urgency and Near-Term Action**: Recognize that climate change is a pressing issue that requires immediate attention. Consider the benefits of taking action in this decade, such as reducing projected losses and damages, delivering co-benefits for air quality and health, and avoiding locked-in high-emissions infrastructure.
2. **Integrating Adaptation and Mitigation**: Understand that climate resilience development integrates both adaptation and mitigation efforts to advance sustainable development. Consider the synergies between these two approaches and how they can support each other in achieving deep emissions reductions and securing a liveable and sustainable future for all.
3. **Equity, Inclusion, and Governance**: Prioritize equity, inclusion, and just transition processes when developing your climate action plan. Recognize that adaptation outcomes are enhanced by in